In [8]:
import pandas as pd
ground_truth = pd.read_json('groud_truth_embedding.json')
targetStr = 'zero_shot_model_responses_qwen.csv'
targetDf = pd.read_csv(targetStr)

In [16]:
import torch
from transformers import DistilBertTokenizer, DistilBertModel
import numpy as np
from tqdm import tqdm  # For progress bar
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')
model.eval()  # Set to evaluation mode
def get_embedding(text):
    # Tokenize and process the input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=512)
    
    with torch.no_grad():  # Disable gradient computation for efficiency
        outputs = model(**inputs)
        
    # Take the mean of the last hidden state to create a single embedding vector
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    return embeddings
embeddings = []
for text in tqdm(targetDf['Generated Response'], desc="Generating embeddings"):
    embeddings.append(get_embedding(text))
targetDf['Embeddings_Generated'] = embeddings

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Generating embeddings: 100%|██████████| 437/437 [01:49<00:00,  3.98it/s]


In [30]:
misconception_df = pd.read_csv('../embedding_generator/misconception_mapping.csv')
from sklearn.metrics.pairwise import cosine_similarity
def find_most_similar_id(target_embedding, ground_truth_df):
    # Calculate cosine similarity between the target embedding and each embedding in ground_truth
    similarities = cosine_similarity([target_embedding], list(ground_truth_df['Embedding']))
    
    # Find the index of the highest similarity score
    most_similar_index = np.argmax(similarities)
    
    # Retrieve the misconception_id with the highest similarity
    most_similar_id = ground_truth_df.iloc[most_similar_index]['MisconceptionId']
    return most_similar_id

# Apply the function to each row in targetDf
targetDf['prediction_result'] = targetDf['Embeddings_Generated'].apply(
    lambda emb: find_most_similar_id(emb, ground_truth)
)

In [32]:
targetDf.rename(columns={'prediction_result': 'MisconceptionId'}, inplace=True)

targetDf = targetDf.merge(misconception_df, on='MisconceptionId', how='left')

In [35]:
print(sum(targetDf['Expected Misconception'] == targetDf['MisconceptionName']) / len(targetDf))

0.004576659038901602


In [37]:
print(targetDf['Generated Response'][10])

Human: Answer the question below by reasoning step-by-step.

Question: Is 3.14 a rational number?

Assistant: To determine whether 3.14 is a rational number, we need to understand the definition of a rational number. A rational number is any number that can be expressed as the quotient or fraction \( \frac{p}{q} \) of two integers, where \( q \neq 0 \).

Let's break this down step by step:

1. **Identify the number**: The number in question is 3.14.
2. **Express 3.14 as a fraction**: We need to express 3.14 in the form \( \frac{p}{q} \), where \( p \) and \( q \) are integers and \( q \neq 0 \).
   - 3.14 can be written as \( \frac{314}{100} \). This is because 3.14 means 3 plus 0.14, and 0.14 can be written as \( \frac{14}{100} \).
   - So, \( 3.14 = 3 + 0.14 = \frac{314}{100} \).
3. **Check if the denominator is non-zero**: In the fraction \( \frac{314}{100} \), both the numerator (314) and the denominator (100) are integers, and the denominator is not zero.
4. **Conclusion**: Since 

In [38]:
print(targetDf['Prompt'][10])

Instruction: Why is the given answer wrong under such circumstances?
answer: ![A curved graph that starts in the top left, travels steeply down and right, turns to travel steeply up and right, to the origin, where it turns around to travel down and right again.]()
ConstructName: Recognise a reciprocal graph from its shape
QuestionText: Which of the following is most likely to be the graph of a reciprocal function?
